In [ ]:

import nfl_data_py as nfl
import pandas as pd
import os
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
import matplotlib.image as mpimg
import tensorflow as tf

import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

import joblib

In [ ]:


'''
Test a deep neural network

Scale features
Define model
Predict
Evaluate
'''


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


model = Sequential([
    Dense(128, input_dim=X_train_s.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae']
)


early_stop = EarlyStopping(patience=20, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(factor=0.5, patience=10)


history = model.fit(
    X_train_s, y_train_s,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, lr_reduce],
    verbose=1
)

# Predict and evaluate on scaled test set
y_pred_nn = model.predict(X_test_s).flatten()

# Metrics
mae = mean_absolute_error(y_test_s, y_pred_nn)
rmse = mean_squared_error(y_test_s, y_pred_nn) ** 0.5
print(f"DNN -> MAE: {mae:.2f}, RMSE: {rmse:.2f}")


'''
Tuned returns: DNN -> MAE: 4.11, RMSE: 5.24
'''

#Default returns DNN -> MAE: 4.12, RMSE: 5.25
'''
history = model.fit(
    X_train_s, y_train_s,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)
'''

In [ ]:
# Plot training & validation loss
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
'''
trying the Keras Tuner
function below goes through first layer, iterates through hidden layers, adds output layer,
then compiles model
'''

def build_model(hp):
    model = Sequential()

    
    model.add(Dense(
        units=hp.Int('units_input', min_value=64, max_value=256, step=32),
        activation='relu',
        input_dim=X_train_s.shape[1]
    ))
    model.add(Dropout(hp.Float('dropout_input', 0.1, 0.5, step=0.1)))

   
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
            activation='relu'
        ))
        model.add(Dropout(hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1)))

    
    model.add(Dense(1))

   
    model.compile(
        optimizer=Adam(
            learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 5e-4, 1e-4])
        ),
        loss='mean_squared_error',
        metrics=['mae']
    )

    return model

'''
this next session runs the keras tuner
'''

tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=10,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='nfl_score_prediction'
)


early_stop = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=10)

tuner.search(
    X_train_s, y_train_s,
    epochs=100,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stop, lr_reduce],
    verbose=1
)

'''
Should return: 

Trial 10 Complete [00h 00m 03s]
val_mae: 4.302618026733398

Best val_mae So Far: 4.150190830230713
'''